In [91]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from warnings import filterwarnings

filterwarnings('ignore')

from modules.data_fetcher import download_historical_data

In [92]:
SYMBOL= 'BTC-USDT'

df = download_historical_data(SYMBOL,'1hour').iloc[-100:]
df["Return"] = df.Close.pct_change()
df.dropna(inplace=True)
print(df.shape)
df.head()

(99, 8)


,Timestamp,Open,Close,High,Low,Amount,Volume,Return
Date,,,,,,,,
2023-02-23 07:00:00,1.677132e+09,24390.5,24383.8,24410.0,24312.9,79.555747,1.938275e+06,-0.000275
2023-02-23 08:00:00,1.677136e+09,24383.8,24374.7,24426.0,24360.7,99.508259,2.427967e+06,-0.000373
2023-02-23 09:00:00,1.677139e+09,24374.7,24405.5,24487.9,24366.2,145.277316,3.548329e+06,0.001264
2023-02-23 10:00:00,1.677143e+09,24405.4,24285.1,24466.6,24270.0,116.489126,2.838676e+06,-0.004933
2023-02-23 11:00:00,1.677146e+09,24283.2,24253.3,24307.6,24200.1,222.009142,5.386005e+06,-0.001309


In [93]:
 
def fft_denoiser(x:np.array, n_components:float=10)-> np.array:
    """Fast fourier transform denoiser (low pass filter)

    Args:
        x (np.array): The data to denoise.
        n_components (float, optional): The value above which the coefficients will be kept. Defaults to 0.0005.

    Returns:
        np.array: The denoised data. 
    """
    n = len(x)
    
    # compute the fft
    fft = np.fft.fft(x, n)
    
    # compute power spectrum density
    # squared magnitud of each fft coefficient
    PSD = fft * np.conj(fft) / n
    
    # keep low frequencies
    _mask = PSD > n_components
    fft = _mask * fft
    
    # inverse fourier transform
    clean_data = np.fft.ifft(fft)
        
    return clean_data.real

In [94]:
df['Close_denoised'] = fft_denoiser(df.Close.values, n_components=100000)

In [95]:
fig = make_subplots(
    rows=1, cols=1# , subplot_titles=("Fourrier and raw data comparing")
)

fig.add_trace(
    go.Scatter(
        name="Raw data",
        x=df.index,
        y=df["Close"],
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        name="Denoised data",
         x=df.index,
         y=df["Close_denoised"],
    ),
    row=1,
    col=1,
)
fig.update_layout(
    xaxis_rangeslider_visible=False,
    showlegend=True,
    title_text="Fourrier extrapolation (prediction)",
)